In [ ]:
import tkinter as tk
from tkinter import messagebox
import import_ipynb
from Router_Heuristic import MazeRouter, CellType

CELL_SIZE = 25
MARGIN = CELL_SIZE

COLOR_MAP = {
    CellType.EMPTY:  "white",
    CellType.BLOCK:  "black",
    CellType.SOURCE: "green",
    CellType.TARGET: "red",
    CellType.VIA:    "yellow",
}

PATH_COLORS = [
    "blue", "orange", "purple", "cyan", "magenta",
    "lime", "pink", "brown", "olive", "teal"
]

class MazeGUI(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Maze Router")

        # Entry frame for file input
        self.entry_frame = tk.Frame(self, padx=10, pady=10)
        self.entry_frame.pack()
        tk.Label(self.entry_frame, text="Config file:").pack(side=tk.LEFT)
        self.filename_entry = tk.Entry(self.entry_frame, width=30)
        self.filename_entry.pack(side=tk.LEFT, padx=(5, 0))
        self.filename_entry.focus()
        self.filename_entry.bind("<Return>", self._on_load)
        self.filename_entry.bind("<Escape>", self._on_exit)

        tk.Button(self.entry_frame, text="Load", command=self._on_load).pack(side=tk.LEFT, padx=5)
        tk.Button(self.entry_frame, text="Exit", command=self._on_exit).pack(side=tk.LEFT, padx=5)

      
        self.grid_container = tk.Frame(self)
        self.grid_canvas = tk.Canvas(self.grid_container)
        self.v_scroll = tk.Scrollbar(self.grid_container, orient=tk.VERTICAL, command=self.grid_canvas.yview)
        self.h_scroll = tk.Scrollbar(self.grid_container, orient=tk.HORIZONTAL, command=self.grid_canvas.xview)
        self.grid_canvas.configure(xscrollcommand=self.h_scroll.set, yscrollcommand=self.v_scroll.set)

        self.v_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        self.h_scroll.pack(side=tk.BOTTOM, fill=tk.X)
        self.grid_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        
        self.grid_frame = tk.Frame(self.grid_canvas)
        self.canvas_window = self.grid_canvas.create_window((0, 0), window=self.grid_frame, anchor="nw")
        self.grid_frame.bind("<Configure>", self._on_frame_configure)

        self.cost_label = tk.Label(self, justify=tk.LEFT, font=("Arial", 10))
        self.back_button = tk.Button(self, text="Back", command=self._on_back)

    def _on_frame_configure(self, event):
     
        self.grid_canvas.configure(scrollregion=self.grid_canvas.bbox("all"))

    def _on_load(self, event=None):
        fname = self.filename_entry.get().strip()
        if not fname:
            messagebox.showwarning("Missing file", "Please enter a config file name.")
            return

        try:
            self.router = MazeRouter(fname)
        except Exception as e:
            messagebox.showerror("Load error", f"Could not load '{fname}':\n{e}")
            return

       
        for w in self.grid_frame.winfo_children():
            w.destroy()
        self.cost_label.pack_forget()
        self.back_button.pack_forget()

        self.entry_frame.pack_forget()
        self.grid_container.pack(fill=tk.BOTH, expand=True)
        self._draw_layers()
        self._draw_costs()
        self.back_button.pack(pady=(10, 20))

    def _on_back(self):
        self.grid_container.pack_forget()
        self.cost_label.pack_forget()
        self.back_button.pack_forget()
        self.entry_frame.pack()

    def _on_exit(self, event=None):
        self.destroy()
        self.quit()

    def _draw_layers(self):
        rows, cols = self.router.rows, self.router.cols
        self.same_results = self.router.route_same_layer_multi_targets()
        self.multi_results = self.router.route_multi_layer()

        self.canvases = {}
        path_colors = {}
        ci = 0
        for idx, _, _ in self.same_results:
            path_colors[("S", idx)] = PATH_COLORS[ci % len(PATH_COLORS)]
            ci += 1
        for idx, *_ in self.multi_results:
            path_colors[("M", idx)] = PATH_COLORS[ci % len(PATH_COLORS)]
            ci += 1

        for layer in range(self.router.NUM_LAYERS):
            lf = tk.LabelFrame(self.grid_frame, text=f"Layer {layer} (M{layer})", padx=5, pady=5)
            lf.grid(row=0, column=layer, padx=10, pady=10)

            width = MARGIN + cols * CELL_SIZE
            height = MARGIN + rows * CELL_SIZE
            c = tk.Canvas(lf, width=width, height=height, bg="white")
            c.pack()
            self.canvases[layer] = c

            for j in range(cols):
                x = MARGIN + j * CELL_SIZE + CELL_SIZE / 2
                c.create_text(x, MARGIN / 2 - 5, text=str(j), font=("Arial", 9))
            for i in range(rows):
                y = MARGIN + i * CELL_SIZE + CELL_SIZE / 2
                c.create_text(MARGIN / 2 - 5, y, text=str(i), font=("Arial", 9))

            for i in range(rows):
                for j in range(cols):
                    cell = self.router.grid[layer][i][j]
                    color = COLOR_MAP.get(cell.kind, "gray")
                    x0 = MARGIN + j * CELL_SIZE
                    y0 = MARGIN + i * CELL_SIZE
                    c.create_rectangle(x0, y0, x0 + CELL_SIZE, y0 + CELL_SIZE,
                                       fill=color, outline="lightgray")
            for i in range(rows + 1):
                y = MARGIN + i * CELL_SIZE
                c.create_line(MARGIN, y, MARGIN + cols * CELL_SIZE, y, fill="lightgray")
            for j in range(cols + 1):
                x = MARGIN + j * CELL_SIZE
                c.create_line(x, MARGIN, x, MARGIN + rows * CELL_SIZE, fill="lightgray")

        for global_idx, (net_idx, segments, _) in enumerate(self.same_results):
            color = path_colors[("S", net_idx)]
            for _, _, path, _ in segments:
                for coord in path:
                    self._fill_cell(coord, color)
            net = self.router.same_layer_nets[net_idx]
            self._label_cell(net[0], f"S{global_idx}")
            for tgt in net[1:]:
                self._label_cell(tgt, f"T{global_idx}")

        base = len(self.same_results)
        for i, (net_idx, src, targets, segment_paths, _) in enumerate(self.multi_results):
            color = path_colors[("M", net_idx)]
            for path in segment_paths:
                for coord in path:
                    self._fill_cell(coord, color)
            self._label_cell(src, f"S{base + i}")
            for tgt in targets:
                self._label_cell(tgt, f"T{base + i}")

    def _draw_costs(self):
        lines = []
        for i, (_, _, total) in enumerate(self.same_results):
            lines.append(f"Net{i} Total Cost: {total}")
        offset = len(self.same_results)
        for i, (_, _, _, _, cost) in enumerate(self.multi_results):
            lines.append(f"Net{offset + i} Total Cost: {cost}")
        self.cost_label.config(text="\n".join(lines))
        self.cost_label.pack(pady=(5, 0), anchor="w")

    def _fill_cell(self, coord, color):
        c = self.canvases[coord.layer]
        x0 = MARGIN + coord.col * CELL_SIZE
        y0 = MARGIN + coord.row * CELL_SIZE
        c.create_rectangle(x0, y0, x0 + CELL_SIZE, y0 + CELL_SIZE,
                           fill=color, outline="lightgray")

    def _label_cell(self, coord, text):
        c = self.canvases[coord.layer]
        x = MARGIN + coord.col * CELL_SIZE + CELL_SIZE / 2
        y = MARGIN + coord.row * CELL_SIZE + CELL_SIZE / 2
        c.create_text(x, y, text=text, font=("Arial", 10, "bold"), anchor="center")

if __name__ == "__main__":
    app = MazeGUI()
    app.mainloop()

: 